In [1]:
import pandas as pd
from catData import getSeasons
from fredapi import Fred
import datetime

In [2]:
FRED_KEY = '6d3ab641594afa0ef4eb9521b8ff6f85'
frd = Fred(api_key=FRED_KEY)
now = datetime.datetime.utcnow()
start = datetime.datetime.strftime(now - datetime.timedelta(days=6971),'%Y-%m-%d')
now = datetime.datetime.strftime(now, '%Y-%m-%d')

In [3]:
ntd = pd.read_excel('./data/Transportation Data Initial.xlsx', sheet_name='Data')
ntd.drop(['datetime', 'State', 'Mode', 'Region', 'Year', 'Month', 'Season'], axis=1, inplace=True)

In [4]:
clean = ntd.groupby(['Transport Type', 'Year Month']).agg(
    {'Unemployment': 'mean',
     'WTI': 'mean',
     'AutoSales': 'mean',
     'RE': 'mean',
     'UPT': 'sum',
     'CPI': 'mean',
     'CPIcn': 'mean',
     'CPIcu': 'mean'}
).reset_index()

In [5]:
clean.columns = ['Mode', 'Date'] + clean.columns.tolist()[2:]
clean['Date'] = pd.to_datetime(clean['Date'], format='%Y%m')
clean.set_index('Date', inplace=True)
clean['Season'] = getSeasons(dates=clean.index)
pop = pd.DataFrame(frd.get_series('POPTHM',
                     observation_start=start,
                     observation_end=now))
pop.columns = ['population']

In [6]:
clean = clean.join(pop, how='inner', on=clean.index).drop('key_0', axis=1)
clean['population'] *= 10e3

In [8]:
clean['pubRate'] = clean['UPT'] / clean['population']
clean['Unemployment'] /= 100

In [10]:
clean.reset_index(inplace=True)
clean.to_csv('./data/final_transportation.csv')